In [6]:
import json_lines
import gzip
import json

from soft404.utils import html2text

with gzip.open('../text_items.jl.gz', 'wt') as outf:
    with json_lines.open_file('../crawler/items.jl.gz') as f:
        for item in json_lines.reader(f, broken=True):
            try:
                text = html2text(item.pop('html'))
            except Exception:
                pass
            else:
                text_item = {
                    'url': item['url'],
                    'text': text,
                    'status': item['status'],
                }
                outf.write(json.dumps(text_item))
                outf.write('\n')